# Aula 2: Análise de Sentimentos usando Bag of Words

Neste notebook iremos treinar um rede de uma única camada para fazer análise de sentimento usando o dataset IMDB.

In [ ]:
nome = 'Arthur Baia'
print(f'Meu nome é {nome}')

Meu nome é Arthur Baia


# Importando as bibliotecas necessárias

In [ ]:
import collections
import pandas as pd
import re
import torch
from typing import List

# Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

--2022-08-31 12:54:03--  http://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.2.19, 104.26.3.19, 172.67.69.159, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/examples/imdb_sample.tgz [following]
--2022-08-31 12:54:03--  https://files.fast.ai/data/examples/imdb_sample.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz’

imdb_sample.tgz     100%[===================>] 558.42K   628KB/s    in 0.9s    

2022-08-31 12:54:04 (628 KB/s) - ‘imdb_sample.tgz’ saved [571827/571827]



Carregamos o dataset .csv usando o pandas:

In [ ]:
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Iremos agora apenas selecionar 100 exemplos de treinamento:

In [ ]:
treino = df[df['is_valid'] == False]  # Apenas treinamento, isto é, descartamos o dataset de validação.

print('treino.shape original:', treino.shape)

treino = treino[:100]  # Aqui truncamos o dataset para os 100 primeiros exemplos. 

print('treino.shape depois:', treino.shape)

treino.shape original: (800, 3)
treino.shape depois: (100, 3)


Iremos dividir este conjunto em entrada (X) e saída desejada (Y, target) e converter as strings "positive" e "negative" do target para valores booleanos:

In [ ]:
X_treino = treino['text']
Y_treino = treino['label']

print(f'Primeiras linhas de X_treino:\n{X_treino.head()}\n')
print(f'Primeiras linhas de Y_treino:\n{Y_treino.head()}\n')

mapeamento = {'positive': True, 'negative': False}
Y_treino = Y_treino.map(mapeamento)
Y_treino = torch.tensor(Y_treino.values, dtype=torch.long)
print(f'Tamanho de Y_treino: {Y_treino.shape}')
print(f'5 primeiras linhas de Y_treino: {Y_treino[:5]}')
print(f'Número de exemplos positivos: {(Y_treino == True).sum()}')
print(f'Número de exemplos negativos: {(Y_treino == False).sum()}')

Primeiras linhas de X_treino:
0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object

Primeiras linhas de Y_treino:
0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object

Tamanho de Y_treino: torch.Size([100])
5 primeiras linhas de Y_treino: tensor([0, 1, 0, 1, 0])
Número de exemplos positivos: 51
Número de exemplos negativos: 49


# Definindo o tokenizador

Agora temos a função de tokenização, isto é, que converte strings para tokens.

In [ ]:
def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """
    # Escreva aqui seu código.
    return re.findall(r"[\w']+", text.lower())

## Testando a função com um exemplo simples


In [ ]:
assert tokenize("I like to eat pizza.") == ['i', 'like', 'to', 'eat', 'pizza'], "Não passou no assert."
print('Passou no assert!')

Passou no assert!


# Definindo o vocabulário

Selecionaremos os `max_tokens` (ex: 1000) tokens mais frequentes do dataset de treino como sendo nosso vocabulário.

In [ ]:
from collections import Counter


In [ ]:
def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """
    # Escreva aqui seu código.
    L = [word for phrase in list(map(tokenize, texts)) for word in phrase]
    k = max_tokens
    vocab = lambda L, k : {value: key for key, value in enumerate(dict(Counter(L).most_common(k)))}
    return vocab(L, k)

## Testando a função


In [ ]:
# Escreva aqui seu(s) assert(s).
teste = ['Fui na padaria e voltei com um pão. Depois fui ao bandejão.']
max_tokens_test = len(teste[0])
assert create_vocab(teste, max_tokens_test) == {'fui': 0, 'na': 1, 'padaria': 2, 'e': 3, 'voltei': 4, 'com': 5, 'um': 6, 'pão': 7, 'depois': 8, 'ao': 9, 'bandejão': 10} 
max_tokens = 1000
vocab = create_vocab(treino['text'],max_tokens)
assert len(vocab) == max_tokens

# Função para converter string para Bag-of-words

In [ ]:
def convert_to_bow(text: str, vocab):
    """
    Returns a bag-of-word vector of size len(vocab).
    """
    # Escreva aqui seu código.
    tokenized = [vocab[token] if token in vocab else -1 for token in tokenize(text)]
    bow = [1 if index in tokenized else 0 for index in torch.arange(len(vocab))]
    return torch.tensor(bow, ).float()

## Testando a função

In [ ]:
# Escreva aqui seu(s) assert(s).
vocab = create_vocab(treino['text'],max_tokens)
assert len(convert_to_bow('the movie is pretty pretty pretty good, it shows the history of vasco da gama!', vocab)) == max_tokens #Cehca se gerou tensor com o tamanho certo
assert torch.equal(convert_to_bow('jeiendkv jeiwhrghij erickmrncje ekcmfmrkjc kjenejxcnenc', vocab), convert_to_bow('jeiwhrghij jeiendkv ekcmfmrkjc erickmrncje kjenejxcnenc', vocab)) #ordem diferente produz mesmo vetor, então one hot funciona
print('passou no assert')

passou no assert


## Definindo a Rede Neural

**Entrada:**

$x \in R^{B \times |V|}$     (bag-of-words)

**Parametros:**

$W \in R^{|V| \times K}$    (weights: matriz de pesos)

$b \in R^{K}$    (bias/viés)

**Saída:**

$p \in R^{B \times K}$  (probabilidade de cada classe)


**Onde:**

$K$ = número de classes

$B$ = tamanho do batch

$|V|$ = tamanho do vocabulário

**Definição da rede:**

$z = xW + b$   (camada linear. $z$ é chamado de logits)

$p_i = \frac{e^{z_i}}{\sum_{j=0}^{K-1} e^{z_j}}$   (softmax)



In [ ]:
class MyModel():

    def __init__(self, dim: int):
        # Escreva seu código aqui.
        self.k = 2 # number of classes
        self.weights = torch.randn((dim,k))*0.001 - 0.0005 #1000x2
        self.weights.requires_grad = True
        self.bias = torch.zeros(k) #2x1
        self.bias.requires_grad = True


    def __call__(self, x): # x = 
        # Escreva seu código aqui.
        z =  torch.matmul(x, self.weights) + self.bias # multiplicação X(B,max_tokens)*W(max_tokens, 2) = Z(B,2) + Biases(2,1)
        max_row = torch.max(z, dim = 1, keepdim=True)
        ez = torch.exp(z - max_row.values)
        softmax = ez / ez.sum(dim = 1, keepdim=True)
        return softmax

## Testando modelo com uma entrada aleatória

Escreva abaixo um pequeno código para testar se seu modelo processa uma matriz de entrada de tamanho `batch_size, dim`, ou seja, a matriz contém `batch_size` exemplos, cada um sendo representado por um vetor de tamanho `dim`.

In [ ]:
# Escreva seu código aqui
model = MyModel(1000)

In [ ]:
vocab = create_vocab(treino['text'],max_tokens)
X = torch.stack([convert_to_bow('the movie is pretty pretty pretty good, it shows the history of vasco da gama!', vocab), 
                 convert_to_bow('the movie is pretty pretty pretty good, curb your enthusiam when watching it!',  vocab), 
                 convert_to_bow('the movie is pretty good, dev patel nails it!', vocab)])
model_tested = model(X)
batch_size = len(X)
assert model_tested.shape == (batch_size, model.k) # Checa se a matriz final tem o shape desejado
assert len(model_tested) == batch_size, "Não passou no assert."
print('Passou nos asserts.')

Passou nos asserts.


# Função de custo Entropia Cruzada

$y \in R^{K}$  (target),

a equação da entropia cruzada associada a um exemplo é dada por:

$L = \sum_{i=0}^{K-1} -y_i \log p_i$   (esta é a loss por exemplo)

Se $y$ for um vetor one-hot (apenas um dos elementos é diferente de zero), podemos simplicar a equação acima para:

$L = -\log p_i$

Onde $i$ é o indice da classe correta. Ou seja, $p_i$ é a probabilidade que o modelo colocou na classe correta.

A função de custo é a **média** da entropia cruzada de cada exemplo no batch.

In [ ]:
def cross_entropy_loss(probs, targets):
    """
    Args:
      probs: a float32 matrix of shape (batch_size, number of classes)
      targets: a long (int64) array of shape (batch_size)

    Returns:
      Mean loss in the batch.
    """
    # Rescreva o código abaixo sem usar laço.
    # batch_size = probs.shape[0]
    # losses = []
    # for i in range(batch_size):
    #   print(targets[i],probs[i, targets[i]], -torch.log(probs[i, targets[i]]))
    #   losses.append(-torch.log(probs[i, targets[i]]))
    
    # losses = torch.stack(losses)
    return (-torch.log(probs[torch.arange(len(targets)), targets])).mean()


## Testando a função entropia cruzada com probabilidades de 50%

Escreva abaixo um pequeno código para testar se a entropia cruzada confere com a resposta do problema 3.6 do exercício da semana passada. Crie um tensor para as probabilidades (50%) e um target também aleatório balanceado e calcule a cross entropia. Qual é o valor esperado da cross entropia nesse caso?

In [ ]:
# escreva seu código aqui

k = 2
batch_size = 1000
probs = torch.ones(batch_size, k)*.5
targets = torch.randint(0, k, (batch_size,))
L = cross_entropy_loss(probs, targets)
assert torch.allclose(torch.log(torch.tensor(k)).data,L) # Garante que o valor esperado é igual ao valor calculado
print('Passou no Assert')

Passou no Assert


# Convertendo dataset de treino para uma matriz de bag-of-words

In [ ]:
vocab = create_vocab(X_treino, max_tokens=1000)
bows = [convert_to_bow(text, vocab) for text in X_treino]
X = torch.stack(bows)
print(X)

tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.]])


# Laço de Treinamento

In [ ]:
num_iterations = 100
learning_rate = 0.1

model = MyModel(dim=len(vocab))

for i in range(num_iterations):
    # Zera os gradientes
    if model.weights.grad is not None:
        model.weights.grad.data.zero_()
        model.bias.grad.data.zero_()

    probs = model(X)
    loss = cross_entropy_loss(probs, Y_treino)
    print(f'iteration: {i}  loss: {loss:.6f}  exp(loss): {torch.exp(loss):.4f}')
    loss.backward()

    #Atualiza os pesos
    model.weights.data -= learning_rate * model.weights.grad.data
    model.bias.data -= model.bias.data - learning_rate * model.bias.grad.data

iteration: 0  loss: 0.691838  exp(loss): 1.9974
iteration: 1  loss: 0.650770  exp(loss): 1.9170
iteration: 2  loss: 0.614112  exp(loss): 1.8480
iteration: 3  loss: 0.581033  exp(loss): 1.7879
iteration: 4  loss: 0.551238  exp(loss): 1.7354
iteration: 5  loss: 0.524212  exp(loss): 1.6891
iteration: 6  loss: 0.499635  exp(loss): 1.6481
iteration: 7  loss: 0.477197  exp(loss): 1.6116
iteration: 8  loss: 0.456642  exp(loss): 1.5788
iteration: 9  loss: 0.437748  exp(loss): 1.5492
iteration: 10  loss: 0.420324  exp(loss): 1.5225
iteration: 11  loss: 0.404207  exp(loss): 1.4981
iteration: 12  loss: 0.389257  exp(loss): 1.4759
iteration: 13  loss: 0.375350  exp(loss): 1.4555
iteration: 14  loss: 0.362382  exp(loss): 1.4367
iteration: 15  loss: 0.350259  exp(loss): 1.4194
iteration: 16  loss: 0.338901  exp(loss): 1.4034
iteration: 17  loss: 0.328238  exp(loss): 1.3885
iteration: 18  loss: 0.318207  exp(loss): 1.3747
iteration: 19  loss: 0.308753  exp(loss): 1.3617
iteration: 20  loss: 0.299827 